# Decision Trees in Practice

* Implement binary decision trees with different early stopping methods.
* Compare models with different stopping parameters.
* Visualize the concept of overfitting in decision trees.


In [1]:
import pandas as pd
import numpy as np

In [2]:
loans = pd.read_csv('lending-club-data.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loans.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


In [4]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x: +1 if x==0 else -1)
loans = loans.drop(['bad_loans'], axis = 1)

In [5]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

# Subsample dataset to be balanced

In [6]:
import json

with open('train-idx.json', 'r') as f:
    train_idx = json.load(f)
    
with open('validation-idx.json', 'r') as f:
    validation_idx = json.load(f)
    


In [7]:
def encoding_binary_feature(data, y_label):
    labels =  data.select_dtypes(include=[object])
    encoded_features =[]
    #encoded_values = np.transpose(np.array(np.ones(len(data),)))
    encoded_values = pd.DataFrame(data[y_label])   
    
    for label in labels:
        
        distinct_features = list(set(data[label].values))
    
        for feature in distinct_features:
            encoded_features.append(str(label + '.'+feature))
            
            #new_array = np.array(np.ones(len(data),))
            encoded_values[encoded_features[-1]] = data[label].apply(lambda x : +1 if x==feature else 0)
    
    return encoded_values

In [8]:
loans_data = encoding_binary_feature(loans,'safe_loans')
print(loans_data.columns.values)

['safe_loans' 'grade.F' 'grade.A' 'grade.B' 'grade.E' 'grade.C' 'grade.G'
 'grade.D' 'term. 36 months' 'term. 60 months' 'home_ownership.OWN'
 'home_ownership.OTHER' 'home_ownership.MORTGAGE' 'home_ownership.RENT'
 'emp_length.9 years' 'emp_length.< 1 year' 'emp_length.n/a'
 'emp_length.7 years' 'emp_length.8 years' 'emp_length.3 years'
 'emp_length.4 years' 'emp_length.5 years' 'emp_length.6 years'
 'emp_length.10+ years' 'emp_length.1 year' 'emp_length.2 years']


## Train_test split

In [9]:
train_data = loans_data.iloc[train_idx].drop(['safe_loans'], axis = 1)
train_Y = loans_data['safe_loans'].iloc[train_idx]

test_data = loans_data.iloc[validation_idx].drop(['safe_loans'], axis = 1)
test_Y = loans_data['safe_loans'].iloc[validation_idx]

In [10]:
safe_loans = (train_Y == 1).sum() + (test_Y ==1).sum()
bad_loans = (train_Y == -1).sum() + (test_Y == -1).sum()
percentage = (safe_loans/(safe_loans + bad_loans)*100)
print('Safe loan percentage: ', percentage)

Safe loan percentage:  50.2236174422


## Decision tree implementation

In this section, we will implement binary decision trees from scratch. There are several steps involved in building a decision tree. For that reason, we have split the entire assignment into several sections.

# Early stopping methods for decision trees

In this section, we will extend the **binary tree implementation** from the previous assignment in order to handle some early stopping conditions. Recall the 3 early stopping methods that were discussed in lecture:

1. Reached a **maximum depth**. (set by parameter `max_depth`).
2. Reached a **minimum node size**. (set by parameter `min_node_size`).
3. Don't split if the **gain in error reduction** is too small. (set by parameter `min_error_reduction`).

For the rest of this assignment, we will refer to these three as **early stopping conditions 1, 2, and 3**.

In [11]:
def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    return True if (data.shape)[0]<=min_node_size else False
    

In [59]:
def error_reduction(error_before_split, error_after_split):
    # Return the error before the split minus the error after the split.
    print("old error: ", error_before_split)
    print("new error: ", error_after_split)
    return (error_before_split - error_after_split)


In [13]:
def intermediate_node_num_mistakes(labels_in_node, key_value):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    if(key_value == -1):
        return (labels_in_node==1).sum()
    elif key_value == 1:
        return (labels_in_node==-1).sum()
    else:
        num_safe_loans = (labels_in_node==1).sum()
    
        # Count the number of -1's (risky loans)
        num_risky_loans = (labels_in_node ==-1).sum()
                
        # Return the number of mistakes that the majority classifier makes.
        return num_safe_loans if num_safe_loans < num_risky_loans else num_risky_loans

In [32]:
def best_splitting_feature(data, features, target):
    
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(target))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        #print(feature)
        # The left split will have all data points where the feature value is 0
        left_split = target[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        right_split =  target[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        left_mistakes =  intermediate_node_num_mistakes(left_split, -1)           

        # Calculate the number of misclassified examples in the right split.
        right_mistakes = intermediate_node_num_mistakes(right_split, 1)      
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        error = (left_mistakes + right_mistakes) / num_data_points
        #print(str(error) + feature)
        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        if error <best_error:
            best_error = error
            best_feature = feature
        
    return best_feature # Return the best feature we found

In [16]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True    }   
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] =          1
    else:
        leaf['prediction'] =          -1
        
    # Return the leaf node        
    return leaf

In [62]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = target
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    

    # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if intermediate_node_num_mistakes(target,0) == 0: 
        print ("Stopping condition 1 reached." )    
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if len(remaining_features) == 0: 
        print ("Stopping condition 2 reached.")    
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth: 
        print ("Reached maximum depth. Stopping for now.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # If the min_size of node reached
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if  reached_minimum_node_size(data, min_node_size): 
        print ("Early stopping condition 2 reached. Reached minimum node size.")
        return create_leaf(target_values)
    
    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    error_before_split = intermediate_node_num_mistakes(target_values, 0) / float(len(data))
    
    splitting_feature = best_splitting_feature(data, remaining_features, target_values)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    left_target = target_values[data[splitting_feature] == 0]
    
    right_split = data[data[splitting_feature] == 1] 
    right_target = target_values[data[splitting_feature] == 1]
    
                                                                                  
    left_mistakes =  intermediate_node_num_mistakes(left_target, -1)           
    right_mistakes = intermediate_node_num_mistakes(right_target, 1)   
    
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    if (error_reduction(error_before_split, error_after_split)) <= min_error_reduction:
        print ("Early stopping condition 3 reached. Minimum error reduction.")
        return create_leaf(target_values)
                                                                                                                                                             
                                                                                  
    #remaining_features =  remaining_features.remove(splitting_feature)
    remaining_features = [x for x in remaining_features if x != splitting_feature]
    
    print ("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(left_target)
    if len(right_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(right_target)

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, left_target, current_depth + 1, max_depth, min_node_size, 
                         min_error_reduction)

    right_tree = decision_tree_create(right_split, remaining_features, right_target, current_depth + 1, max_depth, min_node_size, min_error_reduction) 

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [63]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

In [64]:
train_data.head(2)

,grade.F,grade.A,grade.B,grade.E,grade.C,grade.G,grade.D,term. 36 months,term. 60 months,home_ownership.OWN,...,emp_length.n/a,emp_length.7 years,emp_length.8 years,emp_length.3 years,emp_length.4 years,emp_length.5 years,emp_length.6 years,emp_length.10+ years,emp_length.1 year,emp_length.2 years
1,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0


In [40]:
train_Y.head(2)

1   -1
6   -1
Name: safe_loans, dtype: int64

In [34]:
features = train_data.columns.values

In [68]:
small_decision_tree = decision_tree_create(train_data, features, train_Y, max_depth = 3, 
                                        min_node_size = 0, min_error_reduction=-1)
if count_nodes(small_decision_tree) == 7:
    print ('Test passed!')
else:
    print ('Test failed... try again!')
    print ('Number of nodes found                :', count_nodes(small_decision_tree))
    print ('Number of nodes that should be there : 7' )

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
old error:  0.496346443155
new error:  0.421636578551
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
old error:  0.349235606636
new error:  0.34674184105
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
old error:  0.346305634729
new error:  0.346415259811
Split on feature home_ownership.OTHER. (9119, 3)
--------------------------------------------------------------------
Subtree, depth = 3 (9119 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 3 (3 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth 

## Build a tree!

Now that your code is working, we will train a tree model on the **train_data** with
* `max_depth = 6`
* `min_node_size = 100`, 
* `min_error_reduction = 0.0`

**Warning**: This code block may take a minute to learn. 

In [69]:
my_decision_tree_new = decision_tree_create(train_data, features, train_Y, max_depth = 6, 
                                min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
old error:  0.496346443155
new error:  0.421636578551
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
old error:  0.349235606636
new error:  0.34674184105
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
old error:  0.346305634729
new error:  0.346415259811
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
old error:  0.386138613861
new error:  0.386138613861
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
old error:  0.445484089854
new error

In [70]:
my_decision_tree_old = decision_tree_create(train_data, features, train_Y, max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
old error:  0.496346443155
new error:  0.421636578551
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
old error:  0.349235606636
new error:  0.34674184105
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
old error:  0.346305634729
new error:  0.346415259811
Split on feature home_ownership.OTHER. (9119, 3)
--------------------------------------------------------------------
Subtree, depth = 3 (9119 data points).
old error:  0.346309902402
new error:  0.358701612019
Split on feature emp_length.n/a. (8898, 221)
--------------------------------------------------------------------
Subtree, depth = 4 (8898 data points).
old error:  0.348842436503
new error:  0.362553382783
Split on f

--------------------------------------------------------------------
Subtree, depth = 6 (1380 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 6 (204 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 5 (186 data points).
old error:  0.145161290323
new error:  0.854838709677
Split on feature grade.F. (186, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 4 (977 data points).
old error:  0.205731832139
new error:  0.794268167861
Split on feature grade.F. (977, 0)
Creating leaf node.


## Making predictions

In [52]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [71]:
print ('Predicted class: %s ' % classify(my_decision_tree_new, test_data.iloc[0]))

Predicted class: -1 


In [72]:
classify(my_decision_tree_new, test_data.iloc[0], annotate = True)

Split on term. 36 months = 0
Split on grade.A = 0
At leaf, predicting -1


-1

In [73]:
classify(my_decision_tree_old, test_data.iloc[0], annotate = True)

Split on term. 36 months = 0
Split on grade.A = 0
Split on home_ownership.OTHER = 0
Split on emp_length.n/a = 0
Split on grade.B = 0
Split on emp_length.8 years = 0
At leaf, predicting -1


-1

## Evaluating the model

In [74]:
def evaluate_classification_error(tree, data, target):
    # Apply the classify(tree, x) to each row in your data
    #prediction = data.apply(lambda x: classify(tree, x))
    prediction =[]
    for i in range(len(data)):
        prediction.append(classify(tree, data.iloc[i]))
    
    # Once you've made the predictions, calculate the classification error and return it
    errors = (prediction == target).sum()/len(target)
    return errors

In [75]:
evaluate_classification_error(my_decision_tree_new, test_data, test_Y)


0.57733735458853941

In [76]:
evaluate_classification_error(my_decision_tree_old, test_data, test_Y)


0.61805256355019389

In [77]:
evaluate_classification_error(my_decision_tree_new, train_data, train_Y)


0.57898130238555767

In [78]:
evaluate_classification_error(my_decision_tree_old, train_data, train_Y)

0.61538254889318722

# Exploring the effect of max_depth

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**too small**, **just right**, and **too large**).

Train three models with these parameters:

1. **model_1**: max_depth = 2 (too small)
2. **model_2**: max_depth = 6 (just right)
3. **model_3**: max_depth = 14 (may be too large)

For each of these three, we set `min_node_size = 0` and `min_error_reduction = -1`.

** Note:** Each tree can take up to a few minutes to train. In particular, `model_3` will probably take the longest to train.

In [79]:
model_1 = decision_tree_create(train_data, features, train_Y, max_depth = 2, 
                                min_node_size = 0, min_error_reduction=-1)
model_2 =  decision_tree_create(train_data, features, train_Y, max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)
model_3 =  decision_tree_create(train_data, features, train_Y, max_depth = 14, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
old error:  0.496346443155
new error:  0.421636578551
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
old error:  0.349235606636
new error:  0.34674184105
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
old error:  0.445484089854
new error:  0.461983500589
Split on feature grade.A. (22972, 5029)
--------------------------------------------------------------------
Subtree, depth = 2 (2

Split on feature home_ownership.OWN. (9, 449)
--------------------------------------------------------------------
Subtree, depth = 5 (9 data points).
old error:  0.111111111111
new error:  0.111111111111
Split on feature home_ownership.OTHER. (0, 9)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 5 (449 data points).
old error:  0.24276169265
new error:  0.599109131403
Split on feature emp_length.10+ years. (318, 131)
--------------------------------------------------------------------
Subtree, depth = 6 (318 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 6 (131 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 4 (1824 data points).
old error:  0.287828947368
new error:  0.644188596491
Split on feature emp_length.< 1 year. (1564, 260)
-----------

old error:  0.433155080214
new error:  0.433155080214
Split on feature grade.F. (935, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 6 (79 data points).
old error:  0.481012658228
new error:  0.481012658228
Split on feature term. 60 months. (0, 79)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 4 (221 data points).
old error:  0.244343891403
new error:  0.244343891403
Split on feature grade.G. (215, 6)
--------------------------------------------------------------------
Subtree, depth = 5 (215 data points).
old error:  0.237209302326
new error:  0.237209302326
Split on feature emp_length.9 years. (215, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 5 (6 data points).
old error:  0.5
new error:  0.166666666667
Split on feature home_ownership.OWN. (4, 2)
------------------------------------------------------

Subtree, depth = 12 (916 data points).
old error:  0.482532751092
new error:  0.470524017467
Split on feature emp_length.6 years. (639, 277)
--------------------------------------------------------------------
Subtree, depth = 13 (639 data points).
old error:  0.491392801252
new error:  0.464788732394
Split on feature emp_length.8 years. (466, 173)
--------------------------------------------------------------------
Subtree, depth = 14 (466 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 14 (173 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 13 (277 data points).
old error:  0.42238267148
new error:  0.57761732852
Split on feature grade.F. (277, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 12 (354 data points).
old error:  0.423728813559
n

--------------------------------------------------------------------
Subtree, depth = 8 (235 data points).
old error:  0.268085106383
new error:  0.63829787234
Split on feature emp_length.n/a. (187, 48)
--------------------------------------------------------------------
Subtree, depth = 9 (187 data points).
old error:  0.267379679144
new error:  0.641711229947
Split on feature emp_length.2 years. (150, 37)
--------------------------------------------------------------------
Subtree, depth = 10 (150 data points).
old error:  0.266666666667
new error:  0.64
Split on feature emp_length.6 years. (126, 24)
--------------------------------------------------------------------
Subtree, depth = 11 (126 data points).
old error:  0.277777777778
new error:  0.611111111111
Split on feature emp_length.1 year. (108, 18)
--------------------------------------------------------------------
Subtree, depth = 12 (108 data points).
old error:  0.305555555556
new error:  0.583333333333
Split on feature emp

old error:  0.242537313433
new error:  0.638059701493
Split on feature emp_length.6 years. (646, 158)
--------------------------------------------------------------------
Subtree, depth = 10 (646 data points).
old error:  0.25386996904
new error:  0.609907120743
Split on feature emp_length.9 years. (510, 136)
--------------------------------------------------------------------
Subtree, depth = 11 (510 data points).
old error:  0.274509803922
new error:  0.570588235294
Split on feature emp_length.1 year. (369, 141)
--------------------------------------------------------------------
Subtree, depth = 12 (369 data points).
old error:  0.29539295393
new error:  0.50135501355
Split on feature emp_length.7 years. (224, 145)
--------------------------------------------------------------------
Subtree, depth = 13 (224 data points).
old error:  0.330357142857
new error:  0.40625
Split on feature emp_length.8 years. (111, 113)
--------------------------------------------------------------------


In [80]:
print ("Training data, classification error (model 1):", evaluate_classification_error(model_1, train_data, train_Y))
print ("Training data, classification error (model 2):", evaluate_classification_error(model_2, train_data, train_Y))
print ("Training data, classification error (model 3):", evaluate_classification_error(model_3, train_data, train_Y))

Training data, classification error (model 1): 0.578981302386
Training data, classification error (model 2): 0.615382548893
Training data, classification error (model 3): 0.616269073716


In [82]:
print ("Validation data, classification error (model 1):", evaluate_classification_error(model_1, test_data, test_Y))
print ("Validation data, classification error (model 2):", evaluate_classification_error(model_2, test_data, test_Y))
print ("Validation data, classification error (model 3):", evaluate_classification_error(model_3, test_data, test_Y))

Validation data, classification error (model 1): 0.577337354589
Validation data, classification error (model 2): 0.61805256355
Validation data, classification error (model 3): 0.618806548901


### Measuring the complexity of the tree

Recall in the lecture that we talked about deeper trees being more complex. We will measure the complexity of the tree as

```
  complexity(T) = number of leaves in the tree T
```

Here, we provide a function `count_leaves` that counts the number of leaves in a tree. Using this implementation, compute the number of nodes in `model_1`, `model_2`, and `model_3`. 

In [84]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

In [85]:
print("complexity of model_1: ", count_leaves(model_1))

print("complexity of model_2: ", count_leaves(model_2))

print("complexity of model_3: ", count_leaves(model_3))

complexity of model_1:  4
complexity of model_2:  28
complexity of model_3:  88


# Exploring the effect of min_error

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**negative**, **just right**, and **too positive**).

Train three models with these parameters:
1. **model_4**: `min_error_reduction = -1` (ignoring this early stopping condition)
2. **model_5**: `min_error_reduction = 0` (just right)
3. **model_6**: `min_error_reduction = 5` (too positive)

For each of these three, we set `max_depth = 6`, and `min_node_size = 0`.

** Note:** Each tree can take up to 30 seconds to train.

In [86]:
model_4 = decision_tree_create(train_data, features, train_Y, max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)
model_5 =  decision_tree_create(train_data, features, train_Y, max_depth = 6, 
                                min_node_size = 0, min_error_reduction=0)
model_6 =  decision_tree_create(train_data, features, train_Y, max_depth = 6, 
                                min_node_size = 0, min_error_reduction=5)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
old error:  0.496346443155
new error:  0.421636578551
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
old error:  0.349235606636
new error:  0.34674184105
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
old error:  0.346305634729
new error:  0.346415259811
Split on feature home_ownership.OTHER. (9119, 3)
--------------------------------------------------------------------
Subtree, depth = 3 (9119 data points).
old error:  0.346309902402
new error:  0.358701612019
Split on feature emp_length.n/a. (8898, 221)
--------------------------------------------------------------------
Subtree, depth = 4 (8898 data points).
old error:  0.348842436503
new error:  0.362553382783
Split on f

new error:  0.854838709677
Split on feature grade.F. (186, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 4 (977 data points).
old error:  0.205731832139
new error:  0.794268167861
Split on feature grade.F. (977, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
old error:  0.496346443155
new error:  0.421636578551
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
old error:  0.349235606636
new error:  0.34674184105
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
old error:  0.346305634729
new error:  0.346415259811
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------


In [87]:
print ("Training data, classification error (model 4):", evaluate_classification_error(model_4, train_data, train_Y))
print ("Training data, classification error (model 5):", evaluate_classification_error(model_5, train_data, train_Y))
print ("Training data, classification error (model 6):", evaluate_classification_error(model_6, train_data, train_Y))

Training data, classification error (model 4): 0.615382548893
Training data, classification error (model 5): 0.578981302386
Training data, classification error (model 6): 0.503653556845


In [88]:
print ("Validation data, classification error (model 4):", evaluate_classification_error(model_4, test_data, test_Y))
print ("Validation data, classification error (model 5):", evaluate_classification_error(model_5, test_data, test_Y))
print ("Validation data, classification error (model 6):", evaluate_classification_error(model_6, test_data, test_Y))

Validation data, classification error (model 4): 0.61805256355
Validation data, classification error (model 5): 0.577337354589
Validation data, classification error (model 6): 0.496553209823


In [93]:
print("complexity of model_4: ", count_leaves(model_4))

print("complexity of model_5: ", count_leaves(model_5))

print("complexity of model_6: ", count_leaves(model_6))

complexity of model_4:  28
complexity of model_5:  3
complexity of model_6:  1


# Exploring the effect of min_node_size

We will compare three models trained with different values of the stopping criterion. Again, intentionally picked models at the extreme ends (**too small**, **just right**, and **just right**).

Train three models with these parameters:
1. **model_7**: min_node_size = 0 (too small)
2. **model_8**: min_node_size = 2000 (just right)
3. **model_9**: min_node_size = 50000 (too large)

For each of these three, we set `max_depth = 6`, and `min_error_reduction = -1`.

** Note:** Each tree can take up to 30 seconds to train.

In [89]:
model_7 = decision_tree_create(train_data, features, train_Y, max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)
model_8 =  decision_tree_create(train_data, features, train_Y, max_depth = 6, 
                                min_node_size = 2000, min_error_reduction=-1)
model_9 =  decision_tree_create(train_data, features, train_Y, max_depth = 6, 
                                min_node_size = 50000, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
old error:  0.496346443155
new error:  0.421636578551
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
old error:  0.349235606636
new error:  0.34674184105
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
old error:  0.346305634729
new error:  0.346415259811
Split on feature home_ownership.OTHER. (9119, 3)
--------------------------------------------------------------------
Subtree, depth = 3 (9119 data points).
old error:  0.346309902402
new error:  0.358701612019
Split on feature emp_length.n/a. (8898, 221)
--------------------------------------------------------------------
Subtree, depth = 4 (8898 data points).
old error:  0.348842436503
new error:  0.362553382783
Split on f

Split on feature grade.F. (186, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 4 (977 data points).
old error:  0.205731832139
new error:  0.794268167861
Split on feature grade.F. (977, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
old error:  0.496346443155
new error:  0.421636578551
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
old error:  0.349235606636
new error:  0.34674184105
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
old error:  0.346305634729
new error:  0.346415259811
Split on feature home_ownership.OTHER. (9119, 3)
--------------------------------------------------------------------
Subtree, depth = 3 (9119 data points).


In [90]:
print ("Validation data, classification error (model 7):", evaluate_classification_error(model_7, test_data, test_Y))
print ("Validation data, classification error (model 8):", evaluate_classification_error(model_8, test_data, test_Y))
print ("Validation data, classification error (model 9):", evaluate_classification_error(model_9, test_data, test_Y))

Validation data, classification error (model 7): 0.61805256355
Validation data, classification error (model 8): 0.618375700129
Validation data, classification error (model 9): 0.496553209823


In [91]:
print ("Training data, classification error (model 7):", evaluate_classification_error(model_7, train_data, train_Y))
print ("Training data, classification error (model 8):", evaluate_classification_error(model_8, train_data, train_Y))
print ("Training data, classification error (model 9):", evaluate_classification_error(model_9, train_data, train_Y))

Training data, classification error (model 7): 0.615382548893
Training data, classification error (model 8): 0.615087040619
Training data, classification error (model 9): 0.503653556845


In [92]:
print("complexity of model_7: ", count_leaves(model_7))

print("complexity of model_8: ", count_leaves(model_8))

print("complexity of model_9: ", count_leaves(model_9))

complexity of model_7:  28
complexity of model_8:  17
complexity of model_9:  1
